In [6]:
from utils import *
import pandas as pd
import arcpy
import os
import numpy as np

In [2]:
# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")

In [ ]:
# TAZ feature layer polygons
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
# get as spatial dataframe
sdf_taz = get_fs_data_spatial(taz_url)
# set spatial reference to NAD 1983 UTM Zone 10N
sdf_taz.spatial.sr = sr

In [ ]:
# parcel development layer polygons
parcel_db = sdeCollect + "\\SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

In [7]:
yearly_dev = 100
#Filter the parcels to those that don't have existing development and are zoned residential
sdf_dev_parcels = sdf_units.loc[sdf_units['Residential_Units'] == 0]
sdf_dev_parcels = sdf_dev_parcels.loc[sdf_dev_parcels['OWNERSHIP_TYPE'] == 'Private']
#randomly assign a development priority rank to each parcel
sdf_dev_parcels['DEVPRI'] = np.random.permutation(range(1, len(sdf_dev_parcels)+1))
sdf_dev_parcels=sdf_dev_parcels.sort_values('DEVPRI').reset_index(drop=True)
sdf_dev_parcels['DEV_Year'] = (sdf_dev_parcels.index//yearly_dev)+2023

In [13]:
sdf_dev_parcels_taz = sdf_dev_parcels[['DEV_Year','TAZ','APN']]
TAZ_Development = sdf_dev_parcels_taz.groupby(['DEV_Year','TAZ'])['APN'].count().reset_index()
TAZ_Development.to_csv('TAZ_Development.csv',index=False)